In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [8]:
# Import clusters

data_list =[]

for i in range(8):
    df_temp = pd.read_csv(f"Cluster_{i+1}.csv")
    data_list.append(df_temp)

In [9]:
def Convert_long(df_train):
    df_long = pd.melt(df_train, id_vars=['Index', 'Latitude', 'Longitude'], var_name='Year', value_name='target')
    df_long = df_long.drop(columns=["Latitude", "Longitude"])
    ind_list = list(df_long.iloc[:, 0].values)
    ind_list = list(set(ind_list))
    ind_list.sort()
    df_long_sorted = pd.DataFrame()

    for item in ind_list:
        tempdf = df_long[df_long["Index"] == item]
        tempdf.sort_values(by='Year', inplace=True)
        df_long_sorted = df_long_sorted.append(tempdf, ignore_index=True)
    
    df_long_sorted["Year"] = pd.to_datetime(df_long_sorted["Year"].astype(str), format="%Y")

    train_data = TimeSeriesDataFrame.from_data_frame(
        df_long_sorted,
        id_column="Index",
        timestamp_column="Year")

    return train_data

In [10]:
data_list[0]

,Index,Latitude,Longitude,2010,2011,2012,2013,2014,2015,2016,2017,Vals
0,13,24.66818,72.36657,0.405264,0.146914,0.168368,0.251683,0.180468,0.131804,0.170856,0.177587,0
1,46,24.50698,72.68507,1.185711,0.290858,0.163014,1.161736,0.208937,0.528839,0.536449,0.413165,0
2,66,24.42638,73.08319,6.119396,5.490050,4.758146,5.326427,5.659874,7.116559,6.310906,8.807273,0
3,141,24.18458,73.16282,1.337945,0.465039,1.046371,0.816511,1.148608,1.537456,1.359750,1.382446,0
4,194,24.02338,73.32207,3.236967,2.844021,2.272884,2.228812,0.946102,4.921449,0.822724,2.523114,0
...,...,...,...,...,...,...,...,...,...,...,...,...
195,2408,20.23516,73.32207,13.863707,7.811646,5.893668,9.202684,8.883372,10.242108,10.309837,9.644080,0
196,2409,20.23516,73.40169,7.931598,3.687583,2.922946,3.323191,3.564533,3.614347,4.299843,3.543241,0
197,2410,20.23516,73.48132,0.169694,0.053225,0.042630,0.046837,0.051138,0.051439,0.061234,0.050666,0
198,2416,20.15456,73.08319,2.719220,1.370163,0.818687,1.205721,1.316443,1.324201,1.576338,1.304297,0


In [11]:
def Train_Model(data_list):
    
    for i in range(8):
        X_train = data_list[i].iloc[:,:-1]
        train_data = Convert_long(X_train)
        
        predictor = TimeSeriesPredictor(
        prediction_length=1,
        path=f"forecast_{i+1}",
        target="target",
        eval_metric="MASE")

        predictor.fit(
        train_data,
        presets = "best_quality")
        
        predictions = predictor.predict(train_data)
        predictor.save()
        
        est = (predictions.loc[:,"mean"].values).reshape(-1,1)
        out = pd.DataFrame(est,columns=["Forecast"])
        out.to_csv(f"Forecast_{i+1}.csv",index = False)
        
        print(f"========MODEL_{i+1} DONE============")

In [12]:
Train_Model(data_list)

C:\Users\ATHARVA\AppData\Local\Temp\ipykernel_16308\2024534557.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempdf.sort_values(by='Year', inplace=True)
C:\Users\ATHARVA\AppData\Local\Temp\ipykernel_16308\2024534557.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_long_sorted = df_long_sorted.append(tempdf, ignore_index=True)
c:\users\atharva\appdata\local\programs\python\python39\lib\site-packages\autogluon\timeseries\predictor.py:224: UserWarning: Detected frequency 'AS' is not supported by TimeSeriesPredictor. This may lead to some models not working as intended. Please convert the timestamps to one of the supported frequencies: {'A', 'W', 'T', 'S', 'H', 'M', 'D', 'Y', 'min', 'Q'}. See https://pand

========MODEL_1 DONE============


C:\Users\ATHARVA\AppData\Local\Temp\ipykernel_16308\2024534557.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempdf.sort_values(by='Year', inplace=True)
C:\Users\ATHARVA\AppData\Local\Temp\ipykernel_16308\2024534557.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_long_sorted = df_long_sorted.append(tempdf, ignore_index=True)
c:\users\atharva\appdata\local\programs\python\python39\lib\site-packages\autogluon\timeseries\predictor.py:224: UserWarning: Detected frequency 'AS' is not supported by TimeSeriesPredictor. This may lead to some models not working as intended. Please convert the timestamps to one of the supported frequencies: {'A', 'W', 'T', 'S', 'H', 'M', 'D', 'Y', 'min', 'Q'}. See https://pand

========MODEL_2 DONE============


C:\Users\ATHARVA\AppData\Local\Temp\ipykernel_16308\2024534557.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempdf.sort_values(by='Year', inplace=True)
C:\Users\ATHARVA\AppData\Local\Temp\ipykernel_16308\2024534557.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_long_sorted = df_long_sorted.append(tempdf, ignore_index=True)
c:\users\atharva\appdata\local\programs\python\python39\lib\site-packages\autogluon\timeseries\predictor.py:224: UserWarning: Detected frequency 'AS' is not supported by TimeSeriesPredictor. This may lead to some models not working as intended. Please convert the timestamps to one of the supported frequencies: {'A', 'W', 'T', 'S', 'H', 'M', 'D', 'Y', 'min', 'Q'}. See https://pand

...

========MODEL_6 DONE============


C:\Users\ATHARVA\AppData\Local\Temp\ipykernel_16308\2024534557.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempdf.sort_values(by='Year', inplace=True)
C:\Users\ATHARVA\AppData\Local\Temp\ipykernel_16308\2024534557.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_long_sorted = df_long_sorted.append(tempdf, ignore_index=True)
c:\users\atharva\appdata\local\programs\python\python39\lib\site-packages\autogluon\timeseries\predictor.py:224: UserWarning: Detected frequency 'AS' is not supported by TimeSeriesPredictor. This may lead to some models not working as intended. Please convert the timestamps to one of the supported frequencies: {'A', 'W', 'T', 'S', 'H', 'M', 'D', 'Y', 'min', 'Q'}. See https://pand

========MODEL_7 DONE============


C:\Users\ATHARVA\AppData\Local\Temp\ipykernel_16308\2024534557.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempdf.sort_values(by='Year', inplace=True)
C:\Users\ATHARVA\AppData\Local\Temp\ipykernel_16308\2024534557.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_long_sorted = df_long_sorted.append(tempdf, ignore_index=True)
c:\users\atharva\appdata\local\programs\python\python39\lib\site-packages\autogluon\timeseries\predictor.py:224: UserWarning: Detected frequency 'AS' is not supported by TimeSeriesPredictor. This may lead to some models not working as intended. Please convert the timestamps to one of the supported frequencies: {'A', 'W', 'T', 'S', 'H', 'M', 'D', 'Y', 'min', 'Q'}. See https://pand

========MODEL_8 DONE============


In [1]:
import pandas as pd
import numpy as np

In [2]:
test = pd.read_csv("pred_2018.csv")

In [3]:
test.describe()

,Index,Latitude,Longitude,2010,2011,2012,2013,2014,2015,2016,2017,2018
count,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000
mean,1208.500000,22.506537,71.826215,144.401424,150.860247,109.468899,161.886430,157.954024,132.898633,113.089323,159.163367,147.225205
std,698.160798,1.034936,1.389804,146.506351,142.264612,110.249714,180.337708,139.870466,129.689111,102.092570,158.993069,133.569981
min,0.000000,20.154560,68.624190,0.000167,0.000433,0.000290,0.000532,0.000155,0.000072,0.000151,0.000191,0.000611
25%,604.250000,21.685970,70.774060,23.449163,25.576084,22.711070,27.044554,31.514698,22.732275,22.658495,26.064368,27.046663
50%,1208.500000,22.491970,71.888820,112.298855,126.291077,82.271317,117.662296,136.859337,106.789989,91.721931,120.519817,125.245762
75%,1812.750000,23.297970,73.003570,207.689327,226.197079,163.512547,222.835739,241.965115,196.102222,177.136295,231.067348,222.333919
max,2417.000000,24.668180,74.436820,883.691711,771.061646,871.800415,1232.853760,871.800415,771.061646,514.148987,883.691711,808.984022


In [4]:
for i in range(3,12):
    print(test.iloc[:,i].values.sum())

349162.643347322
364780.078181385
264695.79691034695
391441.386887346
381932.830532243
321348.895277374
273449.981870257
384857.021076038
355990.5464952136
